In [0]:
%pip install prophet
%pip install folium

%restart_python

**Read Tables of Unit 5**

In [0]:
%sql
SELECT * FROM weatherHistory WHERE Unit == 'Unit_5';

-- SELECT result is saved as _sqldf

In [0]:
from prophet import Prophet
import warnings
warnings.filterwarnings("ignore", category=UserWarning)  # for general warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)  # for deprecation

df_temp = _sqldf

# Rename columns using withColumnRenamed
df_temp = df_temp.withColumnRenamed("datetime", "ds").withColumnRenamed("temperature_2m", "y")

# Convert to Pandas DataFrame
df_temp_pd = df_temp.toPandas()

model = Prophet()
model.fit(df_temp_pd)

future = model.make_future_dataframe(periods=2160, freq='H')
forecast = model.predict(future)

display(forecast[["ds", "yhat", "yhat_lower", "yhat_upper"]])

In [0]:
from datetime import datetime, timedelta
now = datetime.now()  # or use timezone-aware datetime if needed
end_time = now + timedelta(hours=23)

forecast_24h = forecast[(forecast['ds'] >= now) & (forecast['ds'] < end_time)]
# Limit to next 24 hours for clarity
# forecast_24h = forecast.tail(2160)

popup_html = """
<b>🌤️ Hourly Forecast (Next 24h)</b><br>
<table style="
    font-size: 12px;
    border-collapse: collapse;
    width: 100%;
    border: 1px solid #ddd;
    border-radius: 6px;
    overflow: hidden;
">
<thead style="background-color: #f2f2f2;">
<tr>
    <th style='padding: 8px; text-align: left; border-bottom: 1px solid #ddd;'>Time</th>
    <th style='padding: 8px; text-align: right; border-bottom: 1px solid #ddd;'>Temp (°C)</th>
    <th style='padding: 8px; text-align: right; border-bottom: 1px solid #ddd;'>↓</th>
    <th style='padding: 8px; text-align: right; border-bottom: 1px solid #ddd;'>↑</th>
</tr>
</thead>
<tbody>
"""

for i, (_, row) in enumerate(forecast_24h.iterrows()):
    background = "#ffffff" if i % 2 == 0 else "#f9f9f9"
    popup_html += f"<tr style='background-color: {background};'>"
    popup_html += f"<td style='padding: 6px; border-bottom: 1px solid #ddd;'>{row['ds'].strftime('%Y-%m-%d %H:%M')}</td>"
    popup_html += f"<td style='padding: 6px; text-align: right; border-bottom: 1px solid #ddd;'>{row['yhat']:.1f}</td>"
    popup_html += f"<td style='padding: 6px; text-align: right; border-bottom: 1px solid #ddd;'>{row['yhat_lower']:.1f}</td>"
    popup_html += f"<td style='padding: 6px; text-align: right; border-bottom: 1px solid #ddd;'>{row['yhat_upper']:.1f}</td>"
    popup_html += "</tr>"

popup_html += "</tbody></table>"



In [0]:
import folium

m = folium.Map(location=[2.8809, 101.352748], zoom_start=10)

folium.Marker(
    [2.8809, 101.352748],
    popup=folium.Popup(popup_html, max_width=500),
    tooltip="Click for forecast",
    tiles="OpenStreetMap",
    icon=folium.Icon(color="black", icon="cloud")
).add_to(m)

m


**Create Table with Forecasted Values**

In [0]:

forecast.rename(columns={'ds': 'datetime', 'yhat': 'forecast_weather', 'yhat_lower': 'forecast_weather_lower', 'yhat_upper': 'forecast_weather_upper'}, inplace=True)
display(forecast)

In [0]:
forecast["Unit"] = "Unit_5"
forecast["Plant"] = "MY0205"
forecast["longitude"] = 101.399143
forecast["latitude"] = 2.9405

forecast["pkey"] = forecast["Unit"] + forecast["datetime"].astype(str)

display(forecast)

In [0]:
spark.createDataFrame(forecast).write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("default.weatherForecast")

In [0]:
%sql
SELECT * FROM weatherforecast ORDER BY datetime DESC